In [1]:
import glob
import tokenize_uk
import os
import re

path = './1551/'

files = glob.glob(path + '*.txt')

In [2]:
f = open(files[0], "r").read()

In [3]:
def find_sentences(text):
    """
    From TEXT select sentences that contain WORD.
    """
    sentences_w_word = []
    paragraphs = tokenize_uk.tokenize_text(text)
    for paragraph in paragraphs:
        for sentence in paragraph:
            if len(sentence) == 1 and sentence[0].isdigit():
                continue
#             for token in sentence:
            sentences_w_word.append(['<s>']+sentence+['</s>'])
    return sentences_w_word

In [4]:
data = []

for filename in os.listdir(path):
    with open(path + filename, "r") as f:
        text = f.read()
    body = text.lower()
    data += find_sentences(body)

print(len(data))

286887


In [5]:
for i in range(len(data)):
    for j in range(len(data[i])):
        if data[i][j].isdigit():
            data[i][j] = '0'

In [6]:
one_gramm = dict()
two_gramm = dict()
three_gramm = dict()

for i in range(len(data)):
    for j in range(len(data[i])):
        key = data[i][j]
        if key in one_gramm:
            one_gramm[key] += 1
        else:
            one_gramm[key] = 1


        if j > 0:
            key = data[i][j-1] + '$' + data[i][j]
            if key in two_gramm:
                two_gramm[key] += 1
            else:
                two_gramm[key] = 1

        if j > 1:
            key = data[i][j-2] + '$' + data[i][j-1] + '$' + data[i][j]
            if key in three_gramm:
                three_gramm[key] += 1
            else:
                three_gramm[key] = 1

In [7]:
len(one_gramm)

237120

In [8]:
len(data)

286887

In [9]:
from collections import Counter

Counter(one_gramm).most_common(10)

[('.', 493249),
 (',', 460756),
 ('<s>', 286887),
 ('</s>', 286887),
 ('0', 210495),
 ('в', 176184),
 ('на', 163613),
 ('не', 140054),
 ('и', 90687),
 ('”', 81601)]

In [10]:
Counter(two_gramm).most_common(10)

[('.$</s>', 158895),
 (',$що', 33706),
 (',$а', 29966),
 ('!$</s>', 26387),
 ('<s>$прошу', 19734),
 ('0$.', 19248),
 ('№$0', 18898),
 (',$0', 17925),
 (',$что', 17897),
 ('0$,', 17551)]

In [11]:
Counter(three_gramm).most_common(10)

[('<s>$добрый$день', 8762),
 ('0$/$0', 8041),
 ('<s>$доброго$дня', 7861),
 ('<s>$0$.', 7647),
 ('по$вул$.', 6925),
 (')$.$</s>', 5681),
 ('0$,$0', 4545),
 ('доброго$дня$!', 4519),
 ('добрый$день$!', 4288),
 ('0$.$</s>', 3954)]

In [12]:
start = dict()

start_symbol = '<s>'
end_symbol = '</s>'

candidates = []
frequencies = []

for (key, freq) in two_gramm.items():
    splitted = key.split('$')
    if splitted[0] == start_symbol:
        start[key] = freq
        candidates.append(splitted[1])
        frequencies.append(freq)

In [13]:
import numpy as np

In [14]:
curr_word = np.random.choice(candidates, 1, p=np.array(frequencies)/np.sum(frequencies))[0]
prev_word = start_symbol

res = prev_word + ' ' + curr_word


# for i in range(20):
counter = 0
while curr_word != end_symbol and counter < 25:
    c = []
    f = []
    
    next_w = dict()

    for (key, freq) in three_gramm.items():
        splitted = key.split('$')
        if splitted[0] == prev_word and splitted[1] == curr_word:
            next_w[key] = freq
            c.append(splitted[2])
            f.append(freq)
    
    if len(c) == 0:
        break
    
    next_word = np.random.choice(c, 1, p=np.array(f)/np.sum(f))
    
    prev_word = curr_word
    curr_word = next_word[0]
    
#     print(curr_word)
    counter += 1
    
    res += (' ' + next_word[0])

print(res)

<s> грузовые машины , машины заезжают на аллею , детскую площадку , щоб запобігти залиттям . а я пропоную , у т . е . это даст
